# <font face="Verdana" size=6 color='#6495ED'>  THIS IS ME

<font face="Verdana" size=3 color='#40E0D0'> Profs. Larissa Driemeier, Thiago Martins

<center><img src='https://drive.google.com/uc?export=view&id=1LYiTAE2KG5dJf_qIoVKOluUhzrK-AmfP' width="600"></center>

Na próximas duas aulas, vocês irão construir um programa para reconhecimento facial de qualquer colega seu do curso de IA. Para isso, serão 3 etapas:
1. Geração de Eigenfaces - utilização do banco de dados *Labeled Faces in the Wild* (LFW) em conjunto com as fotos de seus colegas, para extrair as características principais de uma face humana via PCA;
2. Criação um classificador SVM para reconhecimento facial de seus colegas;
3. Utilização do classificador em tempo real na competição em sala.

Prontos? Este é o __TERCEIRO__ Notebook! Estamos quase lá... Hora da competição

A Webcam captura fotos de colegas e permite que você teste seu algoritmo. Aqui você receberá bastante ajuda, mas terá que automatizar todo o reconhecimento:
- foto é tirada pela webcam;
- foto é cortada, redimensionada, e PCA é aplicado;
- entra no classificador;
- o computador cumprimenta seu colega: __"Olá, Fulano de Tal!"__


In [1]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import matplotlib.pyplot as plt
import html
import pandas as pd
import seaborn as sns
from google.colab import files
import os

# <font face="Verdana" size=6 color='#6495ED'>   Captura de imagem

O código abaixo, em Javascript, foi adaptado do Google Colab Snippets e é utilizado para capturar uma foto da câmera.

In [2]:
def tirar_foto(quality=0.8, texto_botao="Capturar"):
  js = Javascript('''
    async function takePhoto(qual, texto) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = texto;
      div.appendChild(capture);

      // Abre a câmera
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      // Mostra a saída da câmera
      const video = document.createElement('video');
      video.style.display = 'block';
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', qual);
    }
    ''')
  display(js)
  return eval_js('takePhoto({}, "{}")'.format(quality, texto_botao))

A linha de código abaixo utiliza o Haar Cascade, que é um dos mais antigos e poderosos algoritmos de detecção de rosto em tempo real inventados (muito antes do Deep Learning se tornar famoso), e podemos acessá-los com métodos OpenCV. Haar Features não foram usados apenas para detectar rostos, mas também para olhos, lábios, etc. O algoritmo usa recursos de detecção de borda ou linha propostos por Viola e Jones em seu trabalho de pesquisa [Rapid Object Detection using a Boosted Cascade of Simple Features](https://ieeexplore.ieee.org/document/990517) publicado em 2001. Ambas as técnicas ainda serão aprendidas no seu curso de IA.

In [3]:
haar_face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

A função abaixo corte imagens muito grandes...

In [4]:
def crop_square(img, size, interpolation=cv2.INTER_AREA):
    h, w = img.shape[:2]
    min_size = np.amin([h,w])

    # Centralize and crop
    crop_img = img[int(h/2-min_size/2):int(h/2+min_size/2), int(w/2-min_size/2):int(w/2+min_size/2)]
    resized = cv2.resize(crop_img, (size, size), interpolation=interpolation)

    return resized

In [ ]:
try:
  imagem_urlb64 = tirar_foto()
  imbytes = b64decode(imagem_urlb64.split(',')[1])
  im = cv2.imdecode(np.frombuffer(imbytes, dtype=np.uint8), flags=1)
  plt.imshow(im, cmap='gray'),plt.title('Imagem capturada')
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [12]:
gray = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)

Esta é apenas uma observação. Caso você queira fazer upload de uma imagem, os passoa o seguinte código deve substituir a captura de imagem:

* Upload da imagem:
```
uploaded = files.upload()
```

* Para cada rosto detectado, tem-se quatro características `x, y, largura,altura`, isto é, coordenadas `x,y` do canto inferior esquerdo do rosto e `largura, altura` do retângulo que define o rosto. Portanto, a matriz `rostos` definida acima deve ter dimensão `número de rostos detectados` $\times$ 4.
```
name = 'IMG_20220112_173025.jpg'
im = cv2.imread(name)
plt.imshow(im, cmap='gray'),plt.title('Imagem capturada')
#diminuindo a dimensão da imagem
im = crop_square(im, 250)
# Imagem em escala de cinzas
gray = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)
```





# <font face="Verdana" size=6 color='#6495ED'>  Reconhecimento da posição da face
Agora usaremos o Haar Cascade para reconhecimento da imagem.

In [ ]:
# Reconhecimento de faces
rostos = haar_face_cascade.detectMultiScale(im)

print(f'{rostos.shape[0]} rosto(s) detectado(s)')
n_rostos = rostos.shape[0]

print(f'\n A matriz rostos:\n {rostos}')

In [ ]:
# Colocando o retângulo ao redor da face reconhecida
for (x, y, largura, altura) in rostos:
  im_r = cv2.rectangle(im,(x,y),(x+largura,y+altura),(255,0,0),2)
#Plotando a imagem
plt.imshow(im_r, cmap='gray'),plt.title('Rosto(s) Detectado(s)')

In [ ]:
# Zoom no rosto detectado

plt.figure(figsize=(12,12)) # specifying the overall grid size
i=0
for (x, y, largura, altura) in rostos:
  i += 1
  plt.subplot(1,n_rostos,i)    # the number of images in the grid is 5*5 (25)
  plt.imshow(cv2.cvtColor(im[y:y+altura, x:x+largura], cv2.COLOR_BGR2RGB))
  plt.title('image '+ str(i)+'largura: '+str(largura)+'x altura:'+str(altura))
plt.show()

In [ ]:
#se houver vários rostos, escolha o quadro desenhado ao redor da maior imagem,
x, y, largura, altura = rostos[rostos[:,-1].argsort()[-1]] #
print(x,y,largura,altura)
plt.imshow(gray[y:y+altura, x:x+largura], cmap='gray'),plt.title('Rosto Escolhido')

In [ ]:
print(f'A imagem mostrada acima tem {largura} pixels de largura e {altura} pixels de altura \nEssa imagem que deverá ser reconhecida!!!')

Se essa é a imagem que deve ser reconhecida, então precisamos transformá-la para nosso tamanho padrão, ié, `altura_lfw = 125` e  `largura_lfw = 94`

In [18]:
# relação altura vs largura das imagens do do banco de dados LFW
altura_lfw = 62
largura_lfw = 47
razao_aspecto = altura_lfw/largura_lfw
print(f'As imagens tem uma relação altura vs largura de {razao_aspecto}')

As imagens tem uma relação altura vs largura de 1.3191489361702127


In [19]:
def crop_rate(img, x,y,largura,altura, largura_lfw = 94, altura_lfw = 125, interpolation=cv2.INTER_AREA): #cv2.INTER_AREA
      razao_aspecto = altura_lfw/largura_lfw
      centro_x = x + largura/2
      centro_y = y + altura/2
      area = largura*altura
      largura_adj = np.sqrt(area/razao_aspecto)
      altura_adj = razao_aspecto*largura_adj
      x_min = int(np.floor(centro_x-largura_adj/2))
      x_max = int(np.ceil(centro_x+largura_adj/2))
      y_min = int(np.floor(centro_y-altura_adj/2 + 0.5))
      y_max = int(np.ceil(centro_y+altura_adj/2 + 0.5))
      if y_min <0:
         y_max -= y_min
         y_min = 0
      if x_min <0:
         x_max -= x_min
         x_min = 0
      # Centralize and crop
      crop_img = img[y_min:y_max, x_min:x_max]
      img_lfw = cv2.resize(crop_img, (largura_lfw, altura_lfw), interpolation=interpolation)

      print(img_lfw.shape)
      return img_lfw

In [ ]:
Pessoa = crop_rate(gray,x=x,y=y,largura=largura,altura=altura)
plt.imshow(Pessoa, cmap='gray'),plt.title('Imagem para ser reconhecida')

# <font face="Verdana" size=6 color='#6495ED'> Modelo

Não esqueça de carregar seu poderoso classificador e o PCA.

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import pickle as pk

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
pca = pk.load(open('gdrive/MyDrive/AprendizadoMaquinasI/2022/ThisIsMe/PCA/pca.pkl', 'rb'))
model = pk.load(open('gdrive/MyDrive/AprendizadoMaquinasI/2022/ThisIsMe/SVM/model_clf.sav', 'rb'))